In [ ]:
!pip install statsmodels==0.12.1

     |████████████████████████████████| 9.5 MB 15.5 MB/s 
     |████████████████████████████████| 231 kB 81.5 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/datasets/hackprague-dateio/hackprague_txs.csv', sep = ";")
dataSmaller = data.drop(columns=["transaction_id", "tx_date", "merchant_uid", "merchant_category_id", "shop_uid", "shop_type", "country", "client_gender", "client_year_of_birth"])
dataSmaller["client_gender_f"] = data["client_gender"].map({"Male":0.0, "Female":1.0})
dataSmaller["client_age"] = 2020-data["client_year_of_birth"]
del data

/opt/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#dataSmaller.to_csv("smaller.csv")
dataSmaller = pd.read_csv('smaller.csv')
data = dataSmaller
data

/opt/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,client_id,amount,merchant_category,shop_tags,region,client_gender_f,client_age
0,0,146167,-594,Professional Services,{Delivery Service},Pardubice,1.0,40.0
1,1,40161,-986,Professional Services,{Delivery Service},Pardubice,1.0,40.0
2,2,200977,-102,Food And Drink,NaN,Kladno,1.0,55.0
3,3,200977,-158,Food And Drink,NaN,Kladno,1.0,55.0
4,4,200977,-126,Food And Drink,NaN,Kladno,1.0,55.0
...,...,...,...,...,...,...,...,...
13511085,13511085,43128,-126,Freetime,"{Castles And Palaces,Excursions And Tourist At...",Chrudim,0.0,70.0
13511086,13511086,2755,-307,Freetime,"{Castles And Palaces,Excursions And Tourist At...",Chrudim,0.0,40.0
13511087,13511087,54395,-323,Freetime,"{Castles And Palaces,Excursions And Tourist At...",Chrudim,0.0,65.0
13511088,13511088,2073,-262,Freetime,"{Castles And Palaces,Excursions And Tourist At...",Chrudim,0.0,50.0


In [ ]:
data_grouped2 = data.groupby(["client_id", "merchant_category"]).agg({"amount": "sum", "client_age": "mean", "client_gender_f":"mean"}).reset_index()
data_grouped2

,client_id,merchant_category,amount,client_age,client_gender_f
0,27,Atm,-44607,35.0,0.0
1,27,Beauty,-1030,35.0,0.0
2,27,Car,-2893,35.0,0.0
3,27,Children,-2233,35.0,0.0
4,27,Consumer Goods,-44,35.0,0.0
...,...,...,...,...,...
758797,201007,Groceries,-25515,40.0,1.0
758798,201007,Health,-419,40.0,1.0
758799,201007,House And Garden,-679,40.0,1.0
758800,201007,Professional Services,-3975,40.0,1.0


In [ ]:
data_grouped2.describe()

,client_id,amount,client_age,client_gender_f
count,758802.000000,7.588020e+05,758790.000000,758802.000000
mean,89770.473495,-1.176062e+04,38.972041,0.429876
std,55843.186673,4.808070e+04,11.322283,0.495059
min,27.000000,-1.423273e+07,10.000000,0.000000
25%,41852.000000,-1.040000e+04,30.000000,0.000000
50%,84732.000000,-3.134000e+03,40.000000,0.000000
75%,134764.000000,-9.590000e+02,45.000000,1.000000
max,201007.000000,2.653100e+04,90.000000,1.000000


In [ ]:
u_ids = data["client_id"].unique()
categories = [x for x in data["merchant_category"].unique() if isinstance(x, str)]
categories
data[data["merchant_category"] == "Financial Services"]["shop_tags"].unique()

array(['{Insurance}',
       '{Online Financial Services,Shopping Online,Various Financial Services}',
       '{Online Financial Services,Shopping Online,Stock Market Trading}',
       '{Insurance,Online Financial Services,Shopping Online}', '{Bank}',
       '{Loan,Online Financial Services,Shopping Online,Various Financial Services}',
       '{Online Financial Services,Shopping Online}', nan,
       '{Counselling, Consultation,Online Financial Services,Shopping Online,Various Financial Services}',
       '{Cryptocurrencies,Online Financial Services,Shopping Online,Various Financial Services}',
       '{Bank,Online Financial Services,Shopping Online}',
       '{Various Financial Services}',
       '{Insurance,Online Financial Services,Shopping Online,Various Financial Services}',
       '{Online Financial Services,Shopping Online,Stock Market Trading,Various Financial Services}',
       '{Online Digital Services,Shopping Online,Various Digital Services}',
       '{Cryptocurrencies,Onli

In [ ]:
insuranceLovers = data[data["shop_tags"].isin(['{Insurance}',
       '{Insurance,Online Financial Services,Shopping Online}',
       '{Insurance,Online Financial Services,Shopping Online,Various Financial Services}'])]["client_id"].unique()
insuranceLovers

array([123552,   8584,  37245, ...,  24160,  57180,  10602])

In [ ]:
summary_id_cats = data_grouped2.pivot_table('amount', ['client_id',"client_age","client_gender_f"], 'merchant_category').reset_index()
summary_id_cats



merchant_category,client_id,client_age,client_gender_f,Atm,Beauty,Bills And Household,Car,Children,Consumer Goods,Culture,...,Food And Drink,Freetime,Groceries,Health,House And Garden,Pets,Professional Services,Shopping Online,Sport,Travel
0,27,35.0,0.0,-44607.0,-1030.0,NaN,-2893.0,-2233.0,-44.0,NaN,...,-2236.0,-6423.0,-119107.0,-4825.0,NaN,NaN,-24046.0,-1305.0,NaN,-17.0
1,28,25.0,0.0,-31767.0,NaN,NaN,-20412.0,-1810.0,-2991.0,NaN,...,-8862.0,-630.0,-24933.0,-171.0,-8858.0,-686.0,-7248.0,-331.0,NaN,-703.0
2,30,45.0,0.0,-283066.0,NaN,NaN,-17076.0,-205.0,-2261.0,NaN,...,-7098.0,NaN,-78572.0,-1641.0,-40808.0,NaN,-1392.0,NaN,-2611.0,-8437.0
3,33,35.0,0.0,-2970.0,-1220.0,-1200.0,-35123.0,NaN,-2769.0,-798.0,...,-22783.0,-1762.0,-8844.0,-94.0,-7036.0,NaN,-2113.0,-1431.0,-1329.0,-4435.0
4,46,35.0,0.0,-31388.0,-6760.0,NaN,-21252.0,NaN,-485.0,-2795.0,...,-4013.0,-1114.0,-43954.0,-768.0,-13778.0,NaN,NaN,-4428.0,-1866.0,-118.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,200964,55.0,0.0,-34950.0,NaN,NaN,-20180.0,-1429.0,-3655.0,NaN,...,-2851.0,-12519.0,-10308.0,-153.0,-14568.0,NaN,-2978.0,-1657.0,-125.0,NaN
49995,200977,55.0,1.0,-41741.0,NaN,-2910.0,-466.0,-889.0,-1368.0,NaN,...,-3683.0,-3823.0,-82589.0,-104.0,-15307.0,-221.0,-8688.0,-4543.0,-360.0,-1263.0
49996,200993,35.0,1.0,-54870.0,NaN,NaN,-11007.0,NaN,-1513.0,NaN,...,-998.0,-703.0,-34588.0,-93.0,-21226.0,NaN,-573.0,NaN,-3047.0,NaN
49997,200995,55.0,1.0,-31790.0,-3152.0,-58.0,-1963.0,-296.0,-6450.0,-12784.0,...,-7431.0,NaN,-82389.0,-1986.0,-14233.0,NaN,-1469.0,NaN,-28100.0,-4013.0


In [ ]:
from collections import defaultdict
tags_count = defaultdict(int)
for tags_group, c in data["shop_tags"].value_counts().iteritems():
    if isinstance(tags_group, str):
        tgs = tags_group[1:-1].split(",")
        for tg in tgs:
            tags_count[tg] += c
tags_count

defaultdict(int,
            {'Hyper-Supermarket': 3661258,
             'Gas Station': 1168905,
             'Atm-All': 847439,
             'Gambling And Betting': 529888,
             'Online Freetime': 668633,
             'Shopping Online': 2731610,
             'Groceries - Other': 872057,
             'Drugstore': 364847,
             'American Fast Food': 355060,
             'Restaurant Without A Waiter': 530282,
             'News Stand': 239187,
             'Pharmacy': 242123,
             'Advertising / Marketing': 231234,
             'Online Digital Services': 658988,
             'Software / Applications': 468504,
             'Apple': 210076,
             'Hobby Shop': 207086,
             'Fashion Chain - Mix': 267343,
             'Eshop With Everything': 171153,
             'Food - Order': 158904,
             'Online Food And Drink': 169773,
             'Delivery Service': 189889,
             'Local Transport': 268731,
             'Online Travel': 372026,
     

In [ ]:
data["shop_tags"].nunique()

1399

In [ ]:
LOGIT_data = summary_id_cats
LOGIT_data["insurance01"] = summary_id_cats["client_id"].apply(lambda x: int(x in insuranceLovers))
LOGIT_data = LOGIT_data.fillna(0)
LOGIT_data["age_squared"] = LOGIT_data["client_age"].apply(lambda x: x**2)

LOGIT_data["total_spent"] = sum(LOGIT_data[cat] for cat in categories)
for cat in categories:
    LOGIT_data[cat] /= LOGIT_data["total_spent"]
LOGIT_data.to_csv("logit_data.csv")
LOGIT_data

merchant_category,client_id,client_age,client_gender_f,Atm,Beauty,Bills And Household,Car,Children,Consumer Goods,Culture,...,Health,House And Garden,Pets,Professional Services,Shopping Online,Sport,Travel,insurance01,age_squared,total_spent
0,27,35.0,0.0,0.193664,0.004472,-0.000000,0.012560,0.009695,0.000191,-0.000000,...,0.020948,-0.000000,-0.000000,0.104397,0.005666,-0.000000,0.000074,0,1225.0,-230332.0
1,28,25.0,0.0,0.273990,-0.000000,-0.000000,0.176054,0.015611,0.025797,-0.000000,...,0.001475,0.076400,0.005917,0.062514,0.002855,-0.000000,0.006063,0,625.0,-115942.0
2,30,45.0,0.0,0.616684,-0.000000,-0.000000,0.037202,0.000447,0.004926,-0.000000,...,0.003575,0.088904,-0.000000,0.003033,-0.000000,0.005688,0.018381,0,2025.0,-459013.0
3,33,35.0,0.0,0.030652,0.012591,0.012385,0.362485,-0.000000,0.028577,0.008236,...,0.000970,0.072615,-0.000000,0.021807,0.014769,0.013716,0.045771,0,1225.0,-96895.0
4,46,35.0,0.0,0.219798,0.047338,-0.000000,0.148819,-0.000000,0.003396,0.019572,...,0.005378,0.096482,-0.000000,-0.000000,0.031008,0.013067,0.000826,0,1225.0,-142804.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,200964,55.0,0.0,0.284604,-0.000000,-0.000000,0.164330,0.011637,0.029763,-0.000000,...,0.001246,0.118630,-0.000000,0.024250,0.013493,0.001018,-0.000000,0,3025.0,-122802.0
49995,200977,55.0,1.0,0.215286,-0.000000,0.015009,0.002403,0.004585,0.007056,-0.000000,...,0.000536,0.078948,0.001140,0.044810,0.023431,0.001857,0.006514,1,3025.0,-193886.0
49996,200993,35.0,1.0,0.396864,-0.000000,-0.000000,0.079611,-0.000000,0.010943,-0.000000,...,0.000673,0.153523,-0.000000,0.004144,-0.000000,0.022038,-0.000000,0,1225.0,-138259.0
49997,200995,55.0,1.0,0.139273,0.013809,0.000254,0.008600,0.001297,0.028258,0.056007,...,0.008701,0.062355,-0.000000,0.006436,-0.000000,0.123107,0.017581,0,3025.0,-228257.0


In [ ]:
#train test model
LOGIT_data = sm.add_constant(LOGIT_data)
n = LOGIT_data.shape[0]
k = int(n*0.3)
VAR_DROP = []#"Atm", "Fashion", "Beauty", "Electronics", "Drugstore", "Drugstore", "Groceries", "Digital Services"]
VARS = ["const", "total_spent", "client_age", "age_squared", "Children", "Culture", "client_gender_f", "Pets", "Travel", "Bills And Household", "Car", "Consumer Goods", "Financial Services", "Food And Drink", "Freetime", "House And Garden", "Shopping Online", "Sport"]
endog_train = LOGIT_data.iloc[k:n,:]["insurance01"]
exog_train = LOGIT_data.iloc[k:n,:].drop(columns=["client_id","insurance01"]+VAR_DROP)[VARS]
model_2 = sm.Logit(endog = endog_train,exog = exog_train).fit()
model_2.summary()

Optimization terminated successfully.
         Current function value: 0.288563
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            insurance01   No. Observations:                35000
Model:                          Logit   Df Residuals:                    34982
Method:                           MLE   Df Model:                           17
Date:                Sun, 01 Nov 2020   Pseudo R-squ.:                 0.03281
Time:                        00:37:52   Log-Likelihood:                -10100.
converged:                       True   LL-Null:                       -10442.
Covariance Type:            nonrobust   LLR p-value:                1.158e-134
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -4.1914      0.233    -17.959      0.000      -4.649      -3.734
total_spent         -2.566e-07   6.65e-08     -3.860      0.000   -3.87e-07   -1.26e-07
client_age              0.0736      0.011      6.609      0.000       0.052       0.095
age_squared            -0.0008      0.000     -6.319      0.000      -0.001      -0.001
Children               -4.8484      2.146     -2.259      0.024      -9.055      -0.642
Culture                 8.3802      1.760      4.762      0.000       4.931      11.830
client_gender_f        -0.1620      0.043     -3.754      0.000      -0.247      -0.077
Pets                    3.4950      1.285      2.720      0.007       0.977       6.013
Travel                  4.0275      0.331     12.167      0.000       3.379       4.676
Bills And Household    10.0116      1.573      6.365      0.000       6.929      13.095
Car                     1.7938      0.243      7.390      0.000       1.318       2.270
Consumer Goods         -4.8790      0.809     -6.033      0.000      -6.464      -3.294
Financial Services      2.4082      0.207     11.641      0.000       2.003       2.814
Food And Drink         -1.1703      0.364     -3.212      0.001      -1.884      -0.456
Freetime               -0.5260      0.216     -2.436      0.015      -0.949      -0.103
House And Garden        1.0926      0.276      3.956      0.000       0.551       1.634
Shopping Online         2.3719      0.273      8.692      0.000       1.837       2.907
Sport                   2.8107      0.490      5.733      0.000       1.850       3.772
=======================================================================================
"""

In [ ]:
exog_test = LOGIT_data.iloc[:k,:].drop(columns=["client_id","insurance01"]+VAR_DROP)[VARS]
compare_test = LOGIT_data.iloc[:k]["insurance01"]
test_pred = model_2.predict(exog_test)
train_pred = model_2.predict(exog_test)
fpr, tpr, threshold = metrics.roc_curve(compare_test, train_pred)
# Area under curve
roc_auc = metrics.auc(fpr, tpr)
# GINI coefficient
GINI = 2 * (roc_auc - 0.5)
print("Gini=",GINI)
print("AUC=",roc_auc)

train_pred = (train_pred > 0.095).astype(int)
z = compare_test == train_pred
zz = train_pred[compare_test == 0]
print(zz.astype(float).mean())
zz = train_pred[compare_test == 1]
print(zz.astype(float).mean())

print("acc=",sum(z) / len(train_pred))
print("sommersD=",(2*sum(z)-len(train_pred)) / len(train_pred))


Gini= 0.27852136030137453
AUC= 0.6392606801506873
0.39749328558639213
0.5968652037617554
acc= 0.6019067937862524
sommersD= 0.20381358757250484


In [ ]:
rel = data[data["client_id"].isin(insuranceLovers)].groupby(["client_id", "shop_tags"]).agg({"amount": "sum", "client_gender_f": "mean", "client_age":"mean"}).reset_index()
rel

,client_id,shop_tags,amount,client_gender_f,client_age
0,176,"{Administrative Fees,Online Bills And Househol...",-50,1.0,30.0
1,176,"{Advertising / Marketing,Online Digital Servic...",-152,1.0,30.0
2,176,"{Advertising / Marketing,Online Professional S...",-1399,1.0,30.0
3,176,{Atm-All},-152645,1.0,30.0
4,176,"{Car Repair Shop,Spare Parts - Accessories}",-221,1.0,30.0
...,...,...,...,...,...
182346,200977,"{Online Travel,Shopping Online}",-423,1.0,55.0
182347,200977,{Pet Supplies},-221,1.0,55.0
182348,200977,{Pharmacy},-104,1.0,55.0
182349,200977,{Restaurant},-1022,1.0,55.0


In [ ]:
data2 = data.dropna(subset=['shop_tags'])
data2["shop_tags"] = data["shop_tags"].apply(lambda x: x[1:-1].split(",")[0] if isinstance(x, str) else "none")



/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data2

In [ ]:
data_grouped2_2 = data2.groupby(["client_id", "shop_tags"]).agg({"amount": "sum", "client_age": "mean", "client_gender_f":"mean"}).reset_index()
summary_id_tags = data_grouped2_2.pivot_table('amount', ['client_id',"client_age","client_gender_f"], 'shop_tags').reset_index()

LOGIT_data2 = summary_id_tags
LOGIT_data2["insurance01"] = summary_id_cats["client_id"].apply(lambda x: int(x in insuranceLovers))
LOGIT_data2 = LOGIT_data2.fillna(0)
LOGIT_data2["age_squared"] = LOGIT_data2["client_age"].apply(lambda x: x**2)


tagsss = [x for x in data2["shop_tags"].unique() if isinstance(x, str)]
LOGIT_data2["total_spent"] = sum(LOGIT_data2[tag] for tag in tagsss)
for tag in tagsss:
    LOGIT_data2[tag] /= LOGIT_data2["total_spent"]
LOGIT_data2.to_csv("logit_data2.csv")
LOGIT_data2

shop_tags,client_id,client_age,client_gender_f,Accommodation Booking,Accommodations,Administrative Fees,Advertising / Marketing,Aesthetic Medicine,Air Tickets,Airport Lounge,...,Various Digital Services,Various Financial Services,Vending Machine,Veterinary,Webhosting,Wedding,Wine,insurance01,age_squared,total_spent
0,27,35.0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,1225.0,-228942.0
1,28,25.0,0.0,-0.000000,-0.000000,-0.000000,0.005711,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,625.0,-115217.0
2,30,45.0,0.0,-0.000000,0.016612,-0.000000,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,2025.0,-455456.0
3,33,35.0,0.0,0.043709,0.006252,0.013518,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,1225.0,-88768.0
4,46,35.0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,1225.0,-131563.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,200964,55.0,0.0,-0.000000,-0.000000,-0.000000,0.088546,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,3025.0,-121214.0
49995,200977,55.0,1.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1,3025.0,-187006.0
49996,200993,35.0,1.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,1225.0,-135639.0
49997,200995,55.0,1.0,-0.000000,-0.000000,0.000258,-0.000000,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,3025.0,-224913.0


In [ ]:
data2

,Unnamed: 0,client_id,amount,merchant_category,shop_tags,region,client_gender_f,client_age
0,0,146167,-594,Professional Services,{Delivery Service},Pardubice,1.0,40.0
1,1,40161,-986,Professional Services,{Delivery Service},Pardubice,1.0,40.0
60,60,46057,-148,Groceries,{Groceries - Other},Uherské Hradiště,1.0,55.0
61,61,29302,-71,Groceries,{Groceries - Other},Uherské Hradiště,0.0,55.0
62,62,119781,-171,Groceries,{Groceries - Other},Uherské Hradiště,0.0,30.0
...,...,...,...,...,...,...,...,...
13511021,13511021,43723,-252,Groceries,{Hyper-Supermarket},NaN,0.0,45.0
13511022,13511022,43723,-896,Groceries,{Hyper-Supermarket},NaN,0.0,45.0
13511023,13511023,82730,-416,Groceries,{Hyper-Supermarket},NaN,0.0,35.0
13511024,13511024,177827,-3007,Groceries,{Hyper-Supermarket},NaN,0.0,45.0


In [ ]:
#train test model
LOGIT_data = sm.add_constant(LOGIT_data)
n = LOGIT_data.shape[0]
k = int(n*0.15)
VAR_DROP = []#"Atm", "Fashion", "Beauty", "Electronics", "Drugstore", "Drugstore", "Groceries", "Digital Services"]
VARS = ["const", "total_spent", "client_age", "age_squared", "Children", "Culture", "client_gender_f", "Pets", "Travel", "Bills And Household", "Car", "Consumer Goods", "Financial Services", "Food And Drink", "Freetime", "House And Garden", "Shopping Online", "Sport"]
endog_train = LOGIT_data.iloc[k:n,:]["insurance01"]
exog_train = LOGIT_data.iloc[k:n,:].drop(columns=["client_id","insurance01"]+VAR_DROP)[VARS]

model_crt = RandomForestClassifier(n_estimators = 8000, criterion = 'gini', max_depth = 50, min_samples_leaf=3, n_jobs = -1, verbose = 1)
model_crt.fit(X = exog_train, y = endog_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed:  3.2min


KeyboardInterrupt: 

In [ ]:
exog_test = LOGIT_data.iloc[:k,:].drop(columns=["client_id","insurance01"]+VAR_DROP)[VARS]
compare_test = LOGIT_data.iloc[:k]["insurance01"]
test_pred = model_crt.predict(exog_test)
train_pred = model_crt.predict(exog_test)
fpr, tpr, threshold = metrics.roc_curve(compare_test, train_pred)
# Area under curve
roc_auc = metrics.auc(fpr, tpr)
# GINI coefficient
GINI = 2 * (roc_auc - 0.5)
print("Gini=",GINI)
print("AUC=",roc_auc)

train_pred = (train_pred > 0.5).astype(int)
z = compare_test == train_pred
zz = train_pred[compare_test == 0]
print(zz.astype(float).mean())
zz = train_pred[compare_test == 1]
print(zz.astype(float).mean())

print("acc=",sum(z) / len(train_pred))
print("sommersD=",(2*sum(z)-len(train_pred)) / len(train_pred))